# Near Miss Development: V1

In [4]:
# Imports
import numpy as np
from scipy import stats
import pandas as pd
import math
import seaborn as sns
import os

import matplotlib
matplotlib.use('nbagg')
%matplotlib inline

PLACE_HOLDER = None

## Recreate MASS V3 here:

#### Function IO

In [ ]:
def mass_v3(x, y, k):
    """
    Parameters:
    x : Long time series data (numpy array)
    y : Query sequence (numpy array)  
    k : Size of pieces, preferably a power of two (int)
    
    Returns:
    dist : Distance profile (numpy array)
    """

# MASS V3 Function Implementation

In [ ]:
x = PLACE_HOLDER
y = PLACE_HOLDER

In [ ]:
# Length of query and data
m = len(y)
n = len(x)
dist = []

In [ ]:
# Compute statistics for the query y
meany = np.mean(y)
sigmay = np.std(y, ddof=0)

In [ ]:
# Compute moving mean and standard deviation for x
meanx = np.convolve(x, np.ones(m), 'valid') / m
sum_sq = np.convolve(x ** 2, np.ones(m), 'valid')
sigmax = np.sqrt((sum_sq - m * meanx ** 2) / m)

In [ ]:
# Reverse the query and pad with zeros to match size k
y = y[::-1]
y = np.concatenate((y, np.zeros(k - m)))

In [ ]:
# Loop through segments of the time series
for j in range(0, n - k + 1, k - m + 1):
    # Get segment from x and compute FFTs
    X = np.fft.fft(x[j:j + k])
    Y = np.fft.fft(y)

    # Element-wise multiplication in frequency domain and inverse FFT
    Z = X * Y
    z = np.fft.ifft(Z)

    # Compute distance profile for the segment
    d = 2 * (m - (z[m - 1:k] - m * meanx[j:j + k - m + 1] * meany) / (sigmax[j:j + k - m + 1] * sigmay))
    dist.extend(np.sqrt(np.real(d)))

# Handle the last segment if it is longer than the query
j = j + k - m
remaining_length = n - j

if remaining_length >= m:
    # Get the remaining segment from x and adjust query size
    X = np.fft.fft(x[j:])
    y = y[:remaining_length]
    Y = np.fft.fft(y)

    # Element-wise multiplication in frequency domain and inverse FFT
    Z = X * Y
    z = np.fft.ifft(Z)

    # Compute distance profile for the remaining segment
    d = 2 * (m - (z[m - 1:remaining_length] - m * meanx[j:j + remaining_length - m + 1] * meany) / (sigmax[j:j + remaining_length - m + 1] * sigmay))
    dist.extend(np.sqrt(np.real(d)))

# Complete MASS V3 Function


#### My Implementation (Direct Translation from MATLAB MASS V3 Code)

In [ ]:
def mass_v3(x, y, k):
    """
    Parameters:
    x : Long time series data (numpy array)
    y : Query sequence (numpy array)  
    k : Size of pieces, preferably a power of two (int)
    
    Returns:
    dist : Distance profile (numpy array)
    """

    # Length of query and data
    m = len(y)
    n = len(x)
    dist = []
    
    # Compute statistics for the query y
    meany = np.mean(y)
    sigmay = np.std(y, ddof=0)

    # Compute moving mean and standard deviation for x
    meanx = np.convolve(x, np.ones(m), 'valid') / m
    sum_sq = np.convolve(x ** 2, np.ones(m), 'valid')
    sigmax = np.sqrt((sum_sq - m * meanx ** 2) / m)

    # Reverse the query and pad with zeros to match size k
    y = y[::-1]
    y = np.concatenate((y, np.zeros(k - m)))

    # Loop through segments of the time series
    for j in range(0, n - k + 1, k - m + 1):
        # Get segment from x and compute FFTs
        X = np.fft.fft(x[j:j + k])
        Y = np.fft.fft(y)

        # Element-wise multiplication in frequency domain and inverse FFT
        Z = X * Y
        z = np.fft.ifft(Z)

        # Compute distance profile for the segment
        d = 2 * (m - (z[m - 1:k] - m * meanx[j:j + k - m + 1] * meany) / (sigmax[j:j + k - m + 1] * sigmay))
        dist.extend(np.sqrt(np.real(d)))

    # Handle the last segment if it is longer than the query
    j = j + k - m
    remaining_length = n - j
    if remaining_length >= m:
        # Get the remaining segment from x and adjust query size
        X = np.fft.fft(x[j:])
        y = y[:remaining_length]
        Y = np.fft.fft(y)

        # Element-wise multiplication in frequency domain and inverse FFT
        Z = X * Y
        z = np.fft.ifft(Z)

        # Compute distance profile for the remaining segment
        d = 2 * (m - (z[m - 1:remaining_length] - m * meanx[j:j + remaining_length - m + 1] * meany) / (sigmax[j:j + remaining_length - m + 1] * sigmay))
        dist.extend(np.sqrt(np.real(d)))

#### Implementation by Tyler Marrs: [Github Link](https://github.com/tylerwmarrs/mass-ts/blob/master/mass_ts/_mass_ts.py)

In [ ]:
def mass3(ts, query, pieces):
    """
    Compute the distance profile for the given query over the given time 
    series. This version of MASS is hardware efficient given the right number
    of pieces.

    Parameters
    ----------
    ts : array_like
        The array to create a rolling window on.
    query : array_like
        The query.
    pieces : int
        Number of pieces to process. This is best as a power of 2.

    Returns
    -------
    An array of distances.

    Raises
    ------
    ValueError
        If ts is not a list or np.array.
        If query is not a list or np.array.
        If ts or query is not one dimensional.
        If pieces is less than the length of the query.
    """
    ts, query = mtscore.precheck_series_and_query(ts, query)

    m = len(query)
    
    if pieces < m:
        raise ValueError('pieces should be larger than the query length.')
    
    n = len(ts)
    k = pieces
    x = ts
    dist = np.array([])
    
    # compute stats in O(n)
    meany = np.mean(query)
    sigmay = np.std(query)
    
    meanx = mtscore.moving_average(x, m)
    meanx = np.append(np.ones([1, len(x) - len(meanx)]), meanx)
    sigmax = mtscore.moving_std(x, m)
    sigmax = np.append(np.zeros([1, len(x) - len(sigmax)]), sigmax)
    
    # reverse the query and append zeros
    y = np.append(np.flip(query), np.zeros(pieces - m))
    
    step_size = k - m + 1
    stop = n - k + 1
       
    for j in range(0, stop, step_size):
        # The main trick of getting dot products in O(n log n) time
        X = np.fft.fft(x[j:j + k])
        Y = np.fft.fft(y)
        
        Z = X * Y
        z = np.fft.ifft(Z)
            
        d = 2 * (m-(z[m - 1:k] - m * meanx[m + j - 1:j + k] * meany) /
                   (sigmax[m + j - 1:j + k] * sigmay))
        d = np.sqrt(d)
        dist = np.append(dist, d)
   
    j = j + k - m
    k = n - j - 1
    if k >= m:
        X = np.fft.fft(x[j:n-1])
        y = y[0:k]

        Y = np.fft.fft(y)
        Z = X * Y
        z = np.fft.ifft(Z)

        d = 2 * (m-(z[m - 1:k] - m * meanx[j + m - 1:n - 1] * meany) /
                 (sigmax[j + m - 1:n - 1] * sigmay))
       
        d = np.sqrt(d)
        dist = np.append(dist, d)
    
    return np.array(dist)

# Algorithm Benchmarking
- Lets look at the time complexity of each of these functions as a baseline

In [ ]:
# Load the data


In [ ]:
# Calcula